In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.rcParams["figure.figsize"] = (10, 6)

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set it None to display all rows in the dataframe
# pd.set_option('display.max_rows', None)

# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)


# !pip install missingno
import missingno as msno

In [2]:
df_origin = pd.read_csv("clean_scout.csv")
df = df_origin.copy()
df

,make_model,short_description,body_type,price,vat,km,prev_owner,Type,Previous Owners,next_inspection,inspection_new,Warranty,full_service,non_smoking_vehicle,body_color,paint_type,body_color_orig,number_of_doors,number_of_seats,gearing_type,Cylinders,drive_chain,Fuel,co2_emission,emission_class,comfort_convenience,entertainment_media,Extras,safety_security,description,emission_label,Gears,age,hp_kw,Type_,warranty_month,upholstery_type,upholstery_color,displacement_cc,weight_kg,Consumption_comb,Consumption_city,Consumption_country
0,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770.000,VAT deductible,56013.000,2.000,"Used,,Diesel (Particulate Filter)",2,"06/2021,99 g CO2/km (comb)","Yes,Euro 6",4 (Green),NaN,NaN,Black,Metallic,Mythosschwarz,5.000,5.000,Automatic,3.000,front,Diesel,99.000,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...","Sicherheit:, ,Deaktivierung für Beifahrer-Airb...",NaN,NaN,3.000,66.000,Used,NaN,Cloth,Black,1422.000,1220.000,3.800,3.500,3.500
1,Audi A1,1.8 TFSI sport,Sedans,14500.000,Price negotiable,80000.000,NaN,"Used,,Gasoline",NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,3.000,4.000,Automatic,4.000,front,Benzine,129.000,Euro 6,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Sport seats,Sport suspension,Voic...","ABS,Central door lock,Central door lock with r...",Langstreckenfahrzeug daher die hohe Kilometerl...,4 (Green),7.000,2.000,141.000,Used,NaN,Cloth,Grey,1798.000,1255.000,5.600,4.700,4.700
2,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640.000,VAT deductible,83450.000,1.000,"Used,,Diesel (Particulate Filter)",1,NaN,NaN,99 g CO2/km (comb),NaN,NaN,Black,Metallic,mythosschwarz metallic,4.000,4.000,Automatic,NaN,front,Diesel,99.000,Euro 6,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,D...","Fahrzeug-Nummer: AM-95365, ,Ehem. UPE 24.640 E...",4 (Green),NaN,3.000,85.000,Used,NaN,Cloth,Black,1598.000,NaN,3.800,3.400,3.400
3,Audi A1,1.4 TDi Design S tronic,Sedans,14500.000,NaN,73000.000,1.000,"Used,,Diesel (Particulate Filter)",1,NaN,NaN,NaN,99 g CO2/km (comb),Euro 6,Brown,Metallic,NaN,3.000,4.000,Automatic,3.000,NaN,Diesel,99.000,Euro 6,"Air suspension,Armrest,Auxiliary heating,Elect...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport seats,Voice Control","ABS,Alarm system,Central door lock with remote...","Audi A1: ,- 1e eigenaar ,- Perfecte staat: sch...",NaN,6.000,3.000,66.000,Used,NaN,NaN,NaN,1422.000,1195.000,3.800,3.500,3.500
4,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790.000,NaN,16200.000,1.000,"Used,,Diesel (Particulate Filter)",1,NaN,"Yes,109 g CO2/km (comb)",Euro 6,4 (Green),NaN,Black,Metallic,Mythosschwarz Metallic,5.000,5.000,Automatic,3.000,front,Diesel,109.000,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport package,Sport suspension,Vo...","ABS,Central door lock,Driver-side airbag,Elect...","Technik & Sicherheit:,Xenon plus,Klimaautomati...",NaN,NaN,3.000,66.000,Used,NaN,Cloth,Black,1422.000,NaN,4.100,3.800,3.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Blue dCi 200CV EDC Executive 4Control,Van,39950.000,VAT deductible,NaN,NaN,"New,,Diesel (Particulate Filter)",NaN,NaN,NaN,24 months,NaN,NaN,Grey,Metallic,Grigio scuro,5.000,5.000,Automatic,4.000,front,Diesel,139.000,NaN,"Air conditioning,Automatic climate control,Cru...","Bluetooth,Digital radio,Hands-free equipment,O...","Alloy wheels,Touch screen","ABS,Central door lock,Central door lock with r...",

In [3]:
# degiskenlerdeki eksik deger sayisi
df.isnull().sum()

make_model                 0
short_description         46
body_type                 60
price                      0
vat                     4513
km                      1024
prev_owner              6828
Type                       2
Previous Owners         6640
next_inspection        12384
inspection_new         11987
Warranty                6763
full_service            9689
non_smoking_vehicle    12459
body_color               597
paint_type              5772
body_color_orig         3774
number_of_doors          212
number_of_seats          977
gearing_type               0
Cylinders               5680
drive_chain             6858
Fuel                       0
co2_emission            2436
emission_class          3628
comfort_convenience      920
entertainment_media     1374
Extras                  2962
safety_security          982
description              667
emission_label         11974
Gears                   4712
age                     1597
hp_kw                     88
Type_         

In [4]:
# veri setindeki toplam eksik deger sayisi
df.isnull().sum().sum()

162821

In [5]:
# Azalan şekilde sıralamak
df.isnull().sum().sort_values(ascending=False)

non_smoking_vehicle    12459
next_inspection        12384
inspection_new         11987
emission_label         11974
warranty_month         11066
full_service            9689
weight_kg               6974
drive_chain             6858
prev_owner              6828
Warranty                6763
Previous Owners         6640
paint_type              5772
Cylinders               5680
upholstery_color        5022
upholstery_type         4871
Gears                   4712
vat                     4513
body_color_orig         3774
emission_class          3628
Extras                  2962
co2_emission            2436
Consumption_city        2376
Consumption_country     2376
Consumption_comb        2033
age                     1597
entertainment_media     1374
km                      1024
safety_security          982
number_of_seats          977
comfort_convenience      920
description              667
body_color               597
displacement_cc          496
number_of_doors          212
hp_kw         

In [6]:
def summary(dataframe,col_name):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(dropna=False),
           "oran" : (dataframe[col_name].value_counts(normalize=True,dropna=False) * 100)}))
    print("########################################")
    print('Number_of_Nulls   : ', dataframe[col_name].isnull().sum())

In [8]:
na_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
na_cols

['short_description',
 'body_type',
 'vat',
 'km',
 'prev_owner',
 'Type',
 'Previous Owners',
 'next_inspection',
 'inspection_new',
 'Warranty',
 'full_service',
 'non_smoking_vehicle',
 'body_color',
 'paint_type',
 'body_color_orig',
 'number_of_doors',
 'number_of_seats',
 'Cylinders',
 'drive_chain',
 'co2_emission',
 'emission_class',
 'comfort_convenience',
 'entertainment_media',
 'Extras',
 'safety_security',
 'description',
 'emission_label',
 'Gears',
 'age',
 'hp_kw',
 'Type_',
 'warranty_month',
 'upholstery_type',
 'upholstery_color',
 'displacement_cc',
 'weight_kg',
 'Consumption_comb',
 'Consumption_city',
 'Consumption_country']